In [1]:
!pip install dspy-ai

In [2]:
%load_ext autoreload
%autoreload 2
import dspy
#openAi_key = ""

In [3]:
turbo = dspy.OpenAI(model='gpt-4o-mini') #, api_key=openAi_key)
#NECESSÁRIO ALTERAR O RETRIEVER
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
#Define o LLM que será usado, assim não precisa ficar referenciando toda hora
dspy.settings.configure(lm=turbo, rm=colbertv2_wiki17_abstracts)

In [4]:
#Mudar os dados para serem uns exemplos, por exemplo, do Bacen.
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

len(trainset), len(devset)
#Pegar os dados e transforma-los em uma lista assim
# dspy.Examples({'question':'value','answer':'value'})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


(20, 50)

In [5]:
print(f'Tipo do trainset: {type(trainset)}')
print(f'Tipo do item do trainset: {type(trainset[0])}')

Tipo do trainset: <class 'list'>
Tipo do item do trainset: <class 'dspy.primitives.example.Example'>


In [6]:
trainset2 = [dict(x) for x in trainset]

In [7]:
class RAGSignature(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc = 'A short information or factoid that helps answering the question')
    question = dspy.InputField(desc = 'A question to be answered')
    answer = dspy.OutputField(desc = "often between 1 and 5 words")

In [8]:
example_context = 'Search the founding fathers'
example_question = 'Who was the first united states president in the history?'

In [9]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(RAGSignature)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [10]:
from dspy.teleprompt import BootstrapFewShot

# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)

# Compile!
compiled_rag = teleprompter.compile(RAG(), trainset=trainset)

 55%|█████▌    | 11/20 [00:00<00:00, 222.02it/s]

Bootstrapped 4 full traces after 12 examples in round 0.


In [11]:
compiled_rag('Who is the former United States president?')

Prediction(
    context=['Obama (disambiguation) | Barack Obama (born 1961) is a former President of the United States.', 'Joe Biden presidential campaign | Joe Biden, the former Vice President of the United States, has run for the office of President of the United States twice, both unsuccessfully.', 'Lillian Gordy Carter | Bessie Lillian Gordy Carter (August 15, 1898 – October 30, 1983) was the mother of former President of the United States, Jimmy Carter. She was also known for her contribution to nursing in her home state of Georgia and as a Peace Corps volunteer in India as well as writing two books during the Carter presidency.'],
    answer='Barack Obama'
)